In [1]:
import pandas as pd
import numpy as np
import itertools
import random
import json
from sklearn.model_selection import train_test_split
import yaml

In [2]:
params_values_dict = {
    'od_color' : ["Серый", "Белый", "Бледный", "Бледно-Розовый", "Розовый", "Гиперемированный"],
    'od_monotone': ["Наблюдается", "Не наблюдается"],
    'od_size': ["Нормальный", "Больше нормы", "Меньше нормы"],
    'od_shape': ["Правильная", "Овальная", "Не правильная"],
    'od_border': ["Четкие", "Размытые"],
    'od_excavation_size': ["Нормальный", "Больше нормы", "Меньше нормы"],
    'od_excavation_location': ["В центре", "Верхний", "Нижний", "Наружный", "Внутренний", "Верхне-наружный",
                              "Верхне-внутренний", "Нижне-наружный", "Нижне-внутренний"],
    'od_vessels_location': ["В центре", "Верхний", "Нижний", "Наружный", "Внутренний", "Верхне-наружный",
                              "Верхне-внутренний", "Нижне-наружный", "Нижне-внутренний"],
    'od_excavation_ratio': [0.4, 0.9, 0.3, 0.5, 0.7, 0.6, 0.8, 0],
    'od_pathology' : ["К наружи от экскавации темно-серый очаг с четкими границами 0.2 ДД.",
                      "ДЗН бледный с нечеткими границами и расширенной экскавацией до 0,6 ДД.",
                      "На границе ДЗН серовато-белый очаг овальной формы с размытыми краями размером 0,3 ДД.",
                      "ДЗН отечен, выступает в стекловидное тело, границы стерты.",
                      "В височном секторе ДЗН пигментированный очаг коричневого цвета, округлый, размером 0,2 ДД.",
                      "На поверхности ДЗН выявлен точечный белесый очаг диаметром около 0,1 ДД.",
                      "ДЗН имеет дрезообразные отложения на поверхности с четкими контурами.",
                      "В верхне-височном сегменте ДЗН сероватый очаг неправильной формы размером 0,25 ДД.",
                      "Парапапиллярно выявлены мелкие участки атрофии хориоретинальной ткани с четкими границами.",
                      "ДЗН уплощен, с экскавацией до 0,4 ДД, границы четкие, ткани диска истончены.",
                 ],
    'vessels_art_course': ["Нормальный", "Смещение наружу", "Смещение внутрь"],
    'vessels_art_turtuosity': ["Нормальная", "Извитые", "Прямые"],
    'vessels_art_bifurcation': ["Нормальная", "Под острым углом", "Под тупым углом"],
    'vessels_art_caliber': ["Нормальный", "Расширенный", "Суженный"],
    'vessels_vein_course': ["Нормальный", "Смещение наружу", "Смещение внутрь"],
    'vessels_vein_turtuosity': ["Нормальная", "Извитые", "Прямые"],
    'vessels_vein_bifurcation': ["Нормальная", "Под острым углом", "Под тупым углом"],
    'vessels_vein_caliber': ["Нормальный", "Расширенный", "Суженный"],
    'vessels_ratio': ["2:3", "1:3", "4:6", "1:1.5", "1:1"],
    'vessels_pathology': ["По ходу сосудов единичные шарообразные утолщения стенок сосудов, окружённое очагами красного цвета с четкими границами (кровоизлияния)",
                         "Вдоль сосудов видны желтовато-белые отложения (экссудаты) с четкими границами.",
                         "Височные сосудистые аркады деформированы, артериовенозные перекресты с признаками компрессии.",
                         "Извитость венул в области сосудистых аркад с точечными кровоизлияниями.",
                         "Неравномерный калибр сосудов с выраженным сужением артериол в центральной зоне сетчатки.",
                         "Штопорообразная извитость мелких артериол на переферии сетчатки.",
                         "Кистевидные аневризмы артериол сетчатки с единичными точечными кровоизлияниями вокруг.",
                         "Локальные участки облитерации сосудов сетчатки с ишемическими зонами.",
                         "Утолщение стенок сосудов в виде серебристых тяжей (симптом «серебряной проволоки»).",
                         "Сосуды неравномерно сужены, отмечаются зоны прерывания кровотока в виде пустых сегментов.",
                         "Сосуды сетчатки окружены мелкими очагами кровоизлияний, напоминающими «языки пламени».",
                         ],
    'macula_macular_reflex': ["Нормальный", "Сглаженный", "Отсутсвует"],
    'macula_foveal_reflex': ["Нормальный", "Сглаженный", "Отсутсвует"],
    'macula_pathology': ["Дополнительно присутствуют кольцевидные рефлексы (кистовидный отёк).",
                        "Макула отечна, границы нечёткие, присутствует звездчатая фигура из твердых экссудатов.",
                        "В области макулы выявлен крупный беловатый очаг овальной формы с неровными краями размером 0,4 ДД.",
                        "Центр макулы покрыт желтоватыми отложениями (друзы) с четкими контурами.",
                        "Выраженный диффузный отёк макулы, рефлекс отсутствует, присутствуют мелкие точечные кровоизлияния.",
                        "В макуле видны крупные пузырьки (кисты) с прозрачным содержимым.",
                        "Макула содержит пигментированный очаг коричневатого оттенка с четкими границами.",
                        "На периферии макулярной зоны отмечается серовато-белый отёчный вал с четкой границей.",
                        "Центральная ямка сглажена, окружена венчиком из мелких кровоизлияний.",
                        "Макула покрыта крупными кольцевидными отложениями липидного характера (экссудаты)."
                        ],
    'peripheral_pathology': ["По всему глазному дну множественные очаги красного цвета (кровоизлияния).",
                            "По периферии множественные серовато-белые очаги хориоретинальной атрофии с четкими контурами.",
                            "Локальная зона истончения сетчатки с признаками решётчатой дегенерации.",
                            "Обширный участок периферической хориоретинальной атрофии с четкими границами и гиперпигментацией.",
                            "По всей периферии разбросаны мелкие округлые желтоватые очаги (друзы) с чёткими контурами.",
                            "Периферические зоны сетчатки содержат участки тракционных складок с вытянутыми очагами кровоизлияний.",
                            "Единичный очаг отслойки сетчатки на периферии, граница складчатая.",
                            "Периферические кисты сетчатки с серозным содержимым и четкими контурами.",
                            "Периферическая ретиношизис с разделением слоев сетчатки, границы четкие.",
                            "Множественные мелкие пигментированные очаги круглой формы на периферии сетчатки.",
                            "На периферии сетчатки отмечаются единичные очаги отёка с локальными кровоизлияниями вокруг."
                            ]
}

In [3]:
num_combinations = 1
for key in params_values_dict:
    num_combinations *= len(params_values_dict[key])
    
print(f"Всего комбинаций: {num_combinations}")

Всего комбинаций: 1500092133408000


Применяем Latin Hypercube Sampling для подвыборки параметров размером 200 элементов.

Данный подход гарантирует, что каждое значения параметра будет использовано как минимум 1 раз

In [4]:
def lhs_categorical(params_values_dict: dict, n_samples: int, random_state: int = None) -> pd.DataFrame:
    """
    Latin Hypercube Sampling for categorical parameters.
    
    Для каждого параметра создаём n_samples «слоёв»,
    в каждом слое случайно берём точку, а потом
    отображаем дробную позицию [0,1) на индексы категорий.
    
    :param params_values_dict: dict, ключ — имя параметра, значение — список категорий
    :param n_samples: int, число итоговых выборок
    :param random_state: int или None, фиксатор для воспроизводимости
    :return: pd.DataFrame размером (n_samples, len(params_values_dict))
    """
    rng = np.random.default_rng(random_state)
    param_names = list(params_values_dict.keys())
    k = len(param_names)
    
    # матрица дробных позиций: shape = (k, n_samples)
    U = np.zeros((k, n_samples), dtype=float)
    
    for i, name in enumerate(param_names):
        # stratified uniform: (rand + layer_index) / n_samples
        u = (rng.random(n_samples) + np.arange(n_samples)) / n_samples
        # переставим слои в случайном порядке
        rng.shuffle(u)
        U[i, :] = u
    
    # теперь для каждого параметра отображаем u -> индекс категории
    samples = []
    for j in range(n_samples):
        row = {}
        for i, name in enumerate(param_names):
            categories = params_values_dict[name]
            d = len(categories)
            # вычисляем индекс: floor(u * d), но не больше d-1
            idx = min(int(np.floor(U[i, j] * d)), d - 1)
            row[name] = categories[idx]
        samples.append(row)
    
    return pd.DataFrame(samples, columns=param_names)


In [5]:
# генерируем DataFrame
df_train = lhs_categorical(params_values_dict, n_samples=150, random_state=42)
df_test = lhs_categorical(params_values_dict, n_samples=50, random_state=42)

Проверка на упоминания каждого значения параметра:

In [7]:
for column in df_test.columns:
    print(f"{column}: {len(df_test[column].unique())}")

od_color: 6
od_monotone: 2
od_size: 3
od_shape: 3
od_border: 2
od_excavation_size: 3
od_excavation_location: 9
od_vessels_location: 9
od_excavation_ratio: 8
od_pathology: 10
vessels_art_course: 3
vessels_art_turtuosity: 3
vessels_art_bifurcation: 3
vessels_art_caliber: 3
vessels_vein_course: 3
vessels_vein_turtuosity: 3
vessels_vein_bifurcation: 3
vessels_vein_caliber: 3
vessels_ratio: 5
vessels_pathology: 11
macula_macular_reflex: 3
macula_foveal_reflex: 3
macula_pathology: 10
peripheral_pathology: 11


In [8]:
for key in params_values_dict:
    print(f"{key}: {len(params_values_dict[key])}")

od_color: 6
od_monotone: 2
od_size: 3
od_shape: 3
od_border: 2
od_excavation_size: 3
od_excavation_location: 9
od_vessels_location: 9
od_excavation_ratio: 8
od_pathology: 10
vessels_art_course: 3
vessels_art_turtuosity: 3
vessels_art_bifurcation: 3
vessels_art_caliber: 3
vessels_vein_course: 3
vessels_vein_turtuosity: 3
vessels_vein_bifurcation: 3
vessels_vein_caliber: 3
vessels_ratio: 5
vessels_pathology: 11
macula_macular_reflex: 3
macula_foveal_reflex: 3
macula_pathology: 10
peripheral_pathology: 11


## Разбиваем на тренировочную и тестовую выборки

In [9]:
df_train.to_csv('/home/jupyter/datasphere/project/TEST_pathology/train.csv')

In [10]:
df_test.to_csv('/home/jupyter/datasphere/project/TEST_pathology/test.csv')

## Промты для GPT-4.5 для создания обучающего набора текстов

In [11]:
def get_prompt_v2_pathlgy_2(sample_dict):
    json_text = f"""
    {{
    "ДЗН": {{
        "Цвет": "{sample_dict["od_color"]}",
        "Монотонность": "{sample_dict["od_monotone"]}",
        "Размер": "{sample_dict["od_size"]}",
        "Форма": "{sample_dict["od_shape"]}",
        "Границы": "{sample_dict["od_border"]}",
        "Экскавация": {{
            "Размер": {sample_dict["od_excavation_size"]},
            "Сектор": {sample_dict["od_excavation_location"]}
        }},
        "Э/Д": {sample_dict["od_excavation_ratio"]},
        "Сосудистый пучок": {sample_dict["od_vessels_location"]},
        "Патология ДЗН",: {sample_dict["od_pathology"]}, 
    }},
    "Сосуды": {{
        "Артерии": {{
            "Ход": {sample_dict["vessels_art_course"]},
            "Извитость": {sample_dict["vessels_art_turtuosity"]},
            "Бифуркация": {sample_dict["vessels_art_bifurcation"]},
            "Калибр": {sample_dict["vessels_art_caliber"]}
        }},
        "Вены": {{
            "Ход": {sample_dict["vessels_vein_course"]},
            "Извитость": {sample_dict["vessels_vein_turtuosity"]},
            "Бифуркация": {sample_dict["vessels_vein_bifurcation"]},
            "Калибр": {sample_dict["vessels_vein_caliber"]}
        }},
        "А/В индекс": {sample_dict["vessels_ratio"]},
        "Патология Сосудов": {sample_dict["vessels_pathology"]},
    }},
    "Макула": {{
        "Макулярный рефлекс": {sample_dict["macula_macular_reflex"]},
        "Фовеальный рефлекс": {sample_dict["macula_foveal_reflex"]},
        "Патология Макулы": {sample_dict["macula_pathology"]},
    }},
    "Переферия": {{
        "Патология Переферии": {sample_dict["peripheral_pathology"]},
    }}
    }}
    """

    prompt_tmp = f"""**Задача:**
На основе предоставленного JSON-документа с параметрами глазного дна составь развёрнутое текстовое заключение. Ориентируйся на стиль и структуру примера, стремясь к естественной, живой и грамотной формулировке текста.

**Формат:**
- Описание должно быть текстовым, структурированным и последовательным, как в приведённом примере.
- Допускается совместное описание артерий и вен, если у них одинаковые параметры (Пример 1). Если параметры разные, их необходимо описывать отдельно (Пример 2).
- Не надо включать в текстовое описание пояснения из скобок в текстах патологий.

**Данные:**
JSON-документ:
{json_text}

**Пример 1:**
ДЗН бледно-розовый, монотонность не наблюдается, диск нормального размера и правильной формы, границы диска четкие. Экскавация диска нормального размера, расположена в центре, соотношение э/д = 0.3. Сосудистый пучок расположен в центре. ДЗН отечен, выступает в стекловидное тело, границы стерты.
Сосуды имеют нормальный ход и нормальную извитость, бифуркация и калибр в норме, соотношение калибра артерий и вен 2/3. Штопорообразная извитость мелких артериол на переферии сетчатки. 
Макулярный и фовеолярные рефлексы четкие. Макула отечна, границы нечёткие, присутствует звездчатая фигура из твердых экссудатов.
По периферии множественные серовато-белые очаги хориоретинальной атрофии с четкими контурами.

**Пример 2:**
ДЗН серый, монотонный, границы размытые, овальной формы, увеличенного размера. Экскавация расширена, в центре, соотношение э/д = 0.6. Сосудистый пучок расположен центрально. К наружи от экскавации темно-серый очаг с четкими границами 0.2 ДД.
Артерии смещены наружу, извитые, бифуркация под острым углом, калибр расширен. Вены имеют нормальный ход, извитость нормальная, бифуркация нормальная, калибр расширенный, соотношение А/В = 1/1. По ходу сосудов единичные шарообразные утолщения стенок сосудов, окружённое очагами красного цвета с четкими границами. 
Макулярный рефлекс сглаженный, фовеолярный отсутствует. Дополнительно присутствуют кольцевидные рефлексы. 
По всему глазному дну множественные очаги красного цвета.

**Твоя задача:**  
Создай полное текстовое описание глазного дна по JSON-документу, следуй телеграфному стилю, правилам формата и примерам.
"""
    return prompt_tmp

#### train

In [13]:
df_train = pd.read_csv('/home/jupyter/datasphere/project/TRAIN/train.csv')
train_json = df_train.to_dict(orient='records')

In [15]:
GPT_train_prompts = [get_prompt_v2_pathlgy_2(sample) for sample in train_json]

with open("/home/jupyter/datasphere/project/TEST_pathology/GPT/GPT_train_prompts.json", "w", encoding="utf-8") as f:
    json.dump(GPT_train_prompts, f, ensure_ascii=False, indent=4)

#### test

In [16]:
df_test = pd.read_csv('/home/jupyter/datasphere/project/TEST_pathology/test.csv')
test_json = df_test.to_dict(orient='records')

test_prompts = [get_prompt_v2_pathlgy_2(sample) for sample in test_json]

with open("/home/jupyter/datasphere/project/TEST_pathology/test_prompts.json", "w", encoding="utf-8") as f:
    json.dump(test_prompts, f, ensure_ascii=False, indent=4)

## Промты для дообучения YandexGPT

In [2]:
def get_prompt_lora_YGPT(sample_dict):
    json_text = f"""
    {{
    "ДЗН": {{
        "Цвет": "{sample_dict["od_color"]}",
        "Монотонность": "{sample_dict["od_monotone"]}",
        "Размер": "{sample_dict["od_size"]}",
        "Форма": "{sample_dict["od_shape"]}",
        "Границы": "{sample_dict["od_border"]}",
        "Экскавация": {{
            "Размер": {sample_dict["od_excavation_size"]},
            "Сектор": {sample_dict["od_excavation_location"]}
        }},
        "Э/Д": {sample_dict["od_excavation_ratio"]},
        "Сосудистый пучок": {sample_dict["od_vessels_location"]},
        "Патология ДЗН",: {sample_dict["od_pathology"]}, 
    }},
    "Сосуды": {{
        "Артерии": {{
            "Ход": {sample_dict["vessels_art_course"]},
            "Извитость": {sample_dict["vessels_art_turtuosity"]},
            "Бифуркация": {sample_dict["vessels_art_bifurcation"]},
            "Калибр": {sample_dict["vessels_art_caliber"]}
        }},
        "Вены": {{
            "Ход": {sample_dict["vessels_vein_course"]},
            "Извитость": {sample_dict["vessels_vein_turtuosity"]},
            "Бифуркация": {sample_dict["vessels_vein_bifurcation"]},
            "Калибр": {sample_dict["vessels_vein_caliber"]}
        }},
        "А/В индекс": {sample_dict["vessels_ratio"]},
        "Патология Сосудов": {sample_dict["vessels_pathology"]},
    }},
    "Макула": {{
        "Макулярный рефлекс": {sample_dict["macula_macular_reflex"]},
        "Фовеальный рефлекс": {sample_dict["macula_foveal_reflex"]},
        "Патология Макулы": {sample_dict["macula_pathology"]},
    }},
    "Переферия": {{
        "Патология Переферии": {sample_dict["peripheral_pathology"]},
    }}
    }}
    """

    prompt_tmp = f"""**Задача:**
На основе предоставленного JSON-документа с параметрами глазного дна составь его развёрнутое текстовое описание. Cтремись к естественной, живой и грамотной формулировке текста.

**Формат:**
- Описание должно быть текстовым, структурированным и последовательным.
- Допускается совместное описание артерий и вен, если у них одинаковые параметры. Если параметры разные, их необходимо описывать отдельно.
- Запрещено включать в описание пояснения из скобок.
- У тебя должно получиться 4 абзаца.

**Данные:**
JSON-документ:
{json_text}
"""
    return prompt_tmp

In [3]:
df_train = pd.read_csv('/home/jupyter/datasphere/project/TRAIN/train.csv')
train_json = df_train.to_dict(orient='records')

In [4]:
train_texts_path = '/home/jupyter/datasphere/project/TRAIN/GPT/train_texts.json'
with open(train_texts_path, "r", encoding="utf-8") as f:
    train_texts_json = json.load(f)

In [5]:
yandex_train_prompts = [get_prompt_lora_YGPT(sample) for sample in train_json]

In [6]:
df_tmp_1 = pd.DataFrame(yandex_train_prompts, columns=['input'])
df_tmp_2 = pd.DataFrame(train_texts_json)
df_ygpt_train = df_tmp_1.join(df_tmp_2)[['input', 'output_text']]
df_ygpt_train.head()

,input,output_text
0,**Задача:**\nНа основе предоставленного JSON-д...,"ДЗН бледно-розовый, монотонный, уменьшенного р..."
1,**Задача:**\nНа основе предоставленного JSON-д...,"ДЗН белого цвета, монотонный, увеличенного раз..."
2,**Задача:**\nНа основе предоставленного JSON-д...,"ДЗН гиперемированный, монотонность не наблюдае..."
3,**Задача:**\nНа основе предоставленного JSON-д...,"ДЗН гиперемированный, монотонность наблюдается..."
4,**Задача:**\nНа основе предоставленного JSON-д...,"ДЗН гиперемированный, монотонный, уменьшенного..."


In [7]:
df_ygpt_train.to_csv('/home/jupyter/datasphere/project/TRAIN/YGPT_TRAIN_pairs_v2.csv', index=False)

Создаем JSONL/YAML документ для дообучения модели с помощью torchtune

In [8]:
import json

records = [
    {
        "instruction": row["input"],
        "input": "",
        "output": row["output_text"]
    }
    for _, row in df_ygpt_train.iterrows()
]

# Write JSONL
jsonl_path = '/home/jupyter/datasphere/project/TRAIN/ygpt_train_pairs_instruct_v2.jsonl'
with open(jsonl_path, 'w', encoding='utf-8') as f:
    for record in records:
        f.write(json.dumps(record, ensure_ascii=False) + '\n')